In [1]:
imports_path = ".\\imports.ipynb"
tableGAN_path = ".\\tableGAN.ipynb"
utils_path = ".\\utils.ipynb"

import pandas as pd
import os
import pickle
import time
import tensorflow as tf
import shutil
from tqdm.notebook import tqdm
from math import ceil

data_train = pd.read_csv('Datasets\\df_adult_edited_train.csv')
data_test = pd.read_csv('Datasets\\df_adult_edited_test.csv')
#print(data_train.dtypes)
columns_num = data_train.columns[data_train.dtypes == "int64"]

image_dir = ".\\Images"
timer_dir = ".\\Timers"
model_dir = ".\\Model\\tableGAN_ckpt\\compare_adult\\"
timer_path = os.path.join(timer_dir, "timer_tgan.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = Timer()
    timer.save(timer_path)
    del timer
    
from tgan.model import TGANModel

%run "$utils_path"
batch_size = 500
data_train

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,39,Government,77516,Bachelors,13,Never-married,Adm-clerical,White,Male,2174,0,40,US,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married,Exec-managerial,White,Male,0,0,13,US,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,White,Male,0,0,40,US,<=50K
3,53,Private,234721,<=12th,7,Married,Handlers-cleaners,Black,Male,0,0,40,US,<=50K
4,28,Private,338409,Bachelors,13,Married,Prof-specialty,Black,Female,0,0,40,Non-US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,Private,257302,Assoc-acdm,12,Married,Tech-support,White,Female,0,0,38,US,<=50K
30158,40,Private,154374,HS-grad,9,Married,Machine-op-inspct,White,Male,0,0,40,US,>50K
30159,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,White,Female,0,0,40,US,<=50K
30160,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,White,Male,0,0,20,US,<=50K


In [6]:
if retrain:
    tf.reset_default_graph()
    timer_tgan = load_timer(timer_path)
    timer_tgan.start("TGAN")
    tgan = TGANModel(columns_num, output = model_dir + "tgan", 
                     max_epoch = 40, steps_per_epoch = data_train.shape[0] / batch_size, batch_size=batch_size,
                     num_dis_hidden = 256,
                    restore_session = False, save_checkpoints = True)
    tgan.fit(data_train)
    timer_tgan.stop("TGAN")
    timer_tgan.save(timer_path)






[0112 23:29:23 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0112 23:29:23 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0112 23:29:23 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[0112 23:29:23 @registry.py:126] gen/LSTM/00/FC2 input: [500, 100]
[0112 23:29:23 @registry.py:134] gen/LSTM/00/FC2 output: [500, 72]
[0112 23:29:23 @registry.py:126] gen/LSTM/00/FC3 input: [500, 72]
[0112 23:29:23 @registry.py:134] gen/LSTM/00/FC3 output: [500, 100]
[0112 23:29:23 @registry.py:126] gen/LSTM/01/FC input: [500, 100]
[0112 23:29:23 @regist

[0112 23:29:24 @registry.py:134] gen/LSTM/09/FC output: [500, 100]
[0112 23:29:24 @registry.py:126] gen/LSTM/09/FC2 input: [500, 100]
[0112 23:29:24 @registry.py:134] gen/LSTM/09/FC2 output: [500, 118]
[0112 23:29:24 @registry.py:126] gen/LSTM/09/FC3 input: [500, 118]
[0112 23:29:24 @registry.py:134] gen/LSTM/09/FC3 output: [500, 100]
[0112 23:29:24 @registry.py:126] gen/LSTM/10/FC input: [500, 100]
[0112 23:29:24 @registry.py:134] gen/LSTM/10/FC output: [500, 100]
[0112 23:29:24 @registry.py:126] gen/LSTM/10/FC2 input: [500, 100]
[0112 23:29:24 @registry.py:134] gen/LSTM/10/FC2 output: [500, 90]
[0112 23:29:24 @registry.py:126] gen/LSTM/10/FC3 input: [500, 90]
[0112 23:29:24 @registry.py:134] gen/LSTM/10/FC3 output: [500, 100]
[0112 23:29:24 @registry.py:126] gen/LSTM/11/FC input: [500, 100]
[0112 23:29:24 @registry.py:134] gen/LSTM/11/FC output: [500, 100]
[0112 23:29:24 @registry.py:126] gen/LSTM/11/FC2 input: [500, 100]
[0112 23:29:24 @registry.py:134] gen/LSTM/11/FC2 output: [500,

[0112 23:29:31 @base.py:209] Setup callbacks graph ...






[0112 23:29:31 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0112 23:29:31 @summary.py:93] Summarizing collection 'summaries' of size 9.

[0112 23:29:31 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[0112 23:29:32 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0112 23:29:34 @base.py:236] Initializing the session ...
[0112 23:29:34 @base.py:243] Graph Finalized.


[0112 23:29:34 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...

[0112 23:29:34 @base.py:275] Start Epoch 1 ...


 98%|###################################################################################5 |59/60[01:06<00:01, 0.92it/s]

100%|#####################################################################################|60/60[01:09<00:00, 0.86it/s]

...

[0113 00:12:58 @base.py:285] Epoch 39 (global_step 2340) finished, time:1 minute 5 seconds.


[0113 00:12:59 @saver.py:79] Model saved to .\Model\tableGAN_ckpt\compare_adult\tgan\model\model-2340.
[0113 00:12:59 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.92
[0113 00:12:59 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.136
[0113 00:12:59 @monitor.py:467] GAN_loss/discrim/loss: 0.64624
[0113 00:12:59 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.96211
[0113 00:12:59 @monitor.py:467] GAN_loss/gen/klloss: 0.017938
[0113 00:12:59 @monitor.py:467] GAN_loss/gen/loss: 0.94418
[0113 00:12:59 @monitor.py:467] QueueInput/queue_size: 50
[0113 00:12:59 @base.py:275] Start Epoch 40 ...


100%|#####################################################################################|60/60[01:05<00:00, 0.92it/s]

[0113 00:14:04 @base.py:285] Epoch 40 (global_step 2400) finished, time:1 minute 5 seconds.


[0113 00:14:05 @saver.py:79] Model saved to .\Model\tableGAN_ckpt\compare_adult\tgan\model\model-2400.
[0113 00:14:05 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.908
[0113 00:14:05 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.138
[0113 00:14:05 @monitor.py:467] GAN_loss/discrim/loss: 0.64848
[0113 00:14:05 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.96382
[0113 00:14:05 @monitor.py:467] GAN_loss/gen/klloss: 0.024722
[0113 00:14:05 @monitor.py:467] GAN_loss/gen/loss: 0.9391
[0113 00:14:05 @monitor.py:467] QueueInput/queue_size: 50
[0113 00:14:05 @base.py:289] Training has finished!

[0113 00:14:06 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.

[0113 00:14:06 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0113 00:14:06 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)


[0113 00:14:06 @sessinit.py:87] WRN The following variables are in the checkpoint, but not

In [10]:
tgan_synthetic_data_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
tgan_synthetic_data_train.to_csv("Datasets//tgan_synthetic_train_dataset.csv", index=False)
tgan_synthetic_data_test = tgan.sample(ceil(data_test.shape[0] / batch_size) * batch_size).head(data_test.shape[0])
tgan_synthetic_data_test.to_csv("Datasets//tgan_synthetic_test_dataset.csv", index=False)

  6%|#####                                                                               |30/500[00:08<02:05, 3.74it/s]


In [10]:
print(tgan_synthetic_data_train.shape[0], "vs", data_train.shape[0])
print(tgan_synthetic_data_test.shape[0], "vs", data_test.shape[0])

30162 vs 30162
15060 vs 15060


In [2]:
n_synthetic_datasets = 10
dataset_dir = ".//Datasets//"
dataset_gen_dir = os.path.join(dataset_dir, "Generated_for_pred_eval_adult//")
n_synthetic_datasets_existing = 0

In [3]:
n_synthetic_datasets_existing=3
dataset_dir_tgan = os.path.join(dataset_gen_dir, "TGAN") 
model_temp_dir = ".//Model//temp_model"
if os.path.exists(model_temp_dir):
    shutil.rmtree(model_temp_dir)
os.makedirs(dataset_dir_tgan, exist_ok = True)
for i in tqdm(range(n_synthetic_datasets_existing, n_synthetic_datasets), desc = "Generated datasets", disable = True):
    tf.reset_default_graph()
    tgan = TGANModel(columns_num, output = os.path.join(model_temp_dir, f"TGAN{i}"), 
                     max_epoch = 40, steps_per_epoch = data_train.shape[0] / 500, num_dis_hidden = 256,
                     batch_size=batch_size,
                    restore_session=False)
    tgan.fit(data_train.reset_index(drop=True))
    fake_train = tgan.sample(data_train.shape[0])
    fake_train.to_csv(os.path.join(dataset_dir_tgan, f"gen{i}.csv"))
    del tgan





[0114 17:20:09 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0114 17:20:09 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0114 17:20:09 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[0114 17:20:09 @registry.py:126] gen/LSTM/00/FC2 input: [500, 100]
[0114 17:20:09 @registry.py:134] gen/LSTM/00/FC2 output: [500, 72]
[0114 17:20:09 @registry.py:126] gen/LSTM/00/FC3 input: [500, 72]
[0114 17:20:09 @registry.py:134] gen/LSTM/00/FC3 output: [500, 100]
[0114 17:20:09 @registry.py:126] gen/LSTM/01/FC input: [500, 100]
[0114 17:20:09 @regist

[0114 17:20:10 @registry.py:134] gen/LSTM/09/FC output: [500, 100]
[0114 17:20:10 @registry.py:126] gen/LSTM/09/FC2 input: [500, 100]
[0114 17:20:10 @registry.py:134] gen/LSTM/09/FC2 output: [500, 118]
[0114 17:20:10 @registry.py:126] gen/LSTM/09/FC3 input: [500, 118]
[0114 17:20:10 @registry.py:134] gen/LSTM/09/FC3 output: [500, 100]
[0114 17:20:10 @registry.py:126] gen/LSTM/10/FC input: [500, 100]
[0114 17:20:10 @registry.py:134] gen/LSTM/10/FC output: [500, 100]
[0114 17:20:10 @registry.py:126] gen/LSTM/10/FC2 input: [500, 100]
[0114 17:20:10 @registry.py:134] gen/LSTM/10/FC2 output: [500, 90]
[0114 17:20:10 @registry.py:126] gen/LSTM/10/FC3 input: [500, 90]
[0114 17:20:10 @registry.py:134] gen/LSTM/10/FC3 output: [500, 100]
[0114 17:20:10 @registry.py:126] gen/LSTM/11/FC input: [500, 100]
[0114 17:20:10 @registry.py:134] gen/LSTM/11/FC output: [500, 100]
[0114 17:20:10 @registry.py:126] gen/LSTM/11/FC2 input: [500, 100]
[0114 17:20:10 @registry.py:134] gen/LSTM/11/FC2 output: [500,

[0114 17:20:15 @base.py:209] Setup callbacks graph ...






[0114 17:20:15 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0114 17:20:15 @summary.py:93] Summarizing collection 'summaries' of size 9.

[0114 17:20:15 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[0114 17:20:16 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0114 17:20:18 @base.py:236] Initializing the session ...
[0114 17:20:18 @base.py:243] Graph Finalized.


[0114 17:20:18 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...

[0114 17:20:19 @base.py:275] Start Epoch 1 ...


 98%|###################################################################################5 |59/60[01:12<00:01, 0.86it/s]

100%|#####################################################################################|60/60[01:16<00:00, 0.79it/s]

...

[0114 22:43:10 @saver.py:79] Model saved to .//Model//temp_model\TGAN9\model\model-2340.
[0114 22:43:10 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.886
[0114 22:43:10 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.13
[0114 22:43:10 @monitor.py:467] GAN_loss/discrim/loss: 0.65259
[0114 22:43:10 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.96055
[0114 22:43:10 @monitor.py:467] GAN_loss/gen/klloss: 0.028046
[0114 22:43:10 @monitor.py:467] GAN_loss/gen/loss: 0.9325
[0114 22:43:10 @monitor.py:467] QueueInput/queue_size: 50
[0114 22:43:10 @base.py:275] Start Epoch 40 ...


100%|#####################################################################################|60/60[01:07<00:00, 0.89it/s]

[0114 22:44:18 @base.py:285] Epoch 40 (global_step 2400) finished, time:1 minute 7 seconds.


[0114 22:44:18 @saver.py:79] Model saved to .//Model//temp_model\TGAN9\model\model-2400.
[0114 22:44:18 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.902
[0114 22:44:18 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.19
[0114 22:44:18 @monitor.py:467] GAN_loss/discrim/loss: 0.64488
[0114 22:44:18 @monitor.py:467] GAN_loss/gen/final-g-loss: 0.97316
[0114 22:44:18 @monitor.py:467] GAN_loss/gen/klloss: 0.033301
[0114 22:44:18 @monitor.py:467] GAN_loss/gen/loss: 0.93986
[0114 22:44:18 @monitor.py:467] QueueInput/queue_size: 50
[0114 22:44:18 @base.py:289] Training has finished!
[0114 22:44:19 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0114 22:44:20 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0114 22:44:20 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)
[0114 22:44:20 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the grap

 12%|#########9                                                                          |59/500[00:17<02:07, 3.46it/s]
